In [1]:
import pandas as pd
import pandas_ta
import ta

In [2]:
df = pd.read_csv('../BTC_ETH_1week_1m.csv')

In [3]:
def sma(data, window): 
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std*nstd
    lower_band = sma - std*nstd
    return upper_band, lower_band

ta.volatility.AverageTrueRange
symbols = ['ETH']


nstd = 3

for symbol in symbols:
    df[f'{symbol}_SMA'] = sma(df[f'{symbol}-USD_Open'], 20)

    df[f'{symbol}_SMA_200'] = sma(df[f'{symbol}-USD_Open'], 200)

    df[f'{symbol}_Upper_Band'], df[f'{symbol}_Lower_Band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_SMA'], 20, nstd)
    
    df[f'{symbol}_ATR'] = ta.volatility.average_true_range(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], window=14)
    
    dfs = pandas_ta.supertrend(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], period=10, length=None, multiplier=3)
    df[f'{symbol}_SuperDir'] = dfs['SUPERTd_7_3.0']

    df[f'{symbol}_RSI'] = ta.momentum.rsi(close=df[f'{symbol}-USD_Close'], window=20)

df.dropna(inplace=True)

In [4]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, symbols):
        self.short_unit = None
        self.short_amount = 0
        self.long_amount = 0
        self.long_unit = None
        self.short_price = 0
        self.long_price = 0
        
        self.sl = 0
        self.max_inner_buy = .45
        self.band_stop_run = 0.05    # <= 1
        self.slm = 5
        self.leverage = 1



        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.symbols = symbols
        
        self.buys = []
        self.sells = []
        
        self.wins = 0
        self.losses = 0
        self.ties = 0
        
        self.trading_fee_multiplier = 0.9996      # VIP level 0, paying fees with BNB 0.99925
        self.number_of_buys = len(self.buys)

        self.cross_up = {}
        self.cross_down = {}
        self.sma_cross_up = {}
        self.sma_cross_down = {}
        self.reset_crosses([symbols])

    def buy_long(self, symbol, long_price, atr, time):
        self.balance_amount = self.balance_amount * self.trading_fee_multiplier
        self.long_amount = (self.balance_amount * self.leverage / long_price)
        self.long_price = long_price
        self.long_unit = symbol
        self.buys.append([symbol, time, long_price, 'long'])
        self.number_of_buys = len(self.buys)
        self.sl = long_price - (self.slm*atr)
        print('\n\n', 'filled long', long_price, time)

    def buy_short(self, symbol, short_price, atr, time):
        self.balance_amount = self.balance_amount * self.trading_fee_multiplier
        self.short_amount = (self.balance_amount * self.leverage / short_price)
        self.short_price = short_price
        self.short_unit = symbol
        self.buys.append([symbol, time, short_price, 'short'])
        self.number_of_buys = len(self.buys)
        self.sl = short_price + (self.slm*atr)
        print('\n\n', 'filled short', short_price, time)

    def sell_long(self, sell_price, time):
        self.balance_amount = self.long_amount * sell_price * self.trading_fee_multiplier / self.leverage
        self.sells.append([symbol, time, sell_price])
        self.long_unit = None
        self.long_amount = 0
    
    def sell_short(self, sell_price, time):
        self.balance_amount = (self.balance_amount + ((self.short_amount * ((self.short_price)*self.trading_fee_multiplier - sell_price)) / self.leverage))
        self.sells.append([symbol, time, sell_price])
        self.short_unit = None
        self.short_amount = 0
    

    def reset_crosses(self, symbols):
        for symbol in self.symbols:
            self.cross_up[symbol] = False
            self.cross_down[symbol] = False
            self.sma_cross_up[symbol] = False
            self.sma_cross_down[symbol] = False
    
    def good_buy(self, type, bbhigh, bblow, close):
        bbrange = bbhigh - bblow
        maximum = 0
        if type == 'long':
            maximum = bblow + (bbrange * self.max_inner_buy)
        
        if type == 'short':
            maximum = bbhigh - (bbrange * self.max_inner_buy)
        
        return maximum



In [5]:
"""
BUY:
    Long: 
        1. Check if symbol crosses under lower boll. band
        2. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Buy long

    Short:
        1. Check if symbol crosses over upper boll. band
        2. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Buy short

SELL:
    Long:
        1. Check if symbol crosses over upper boll. band
        2. Check if close is under stop loss; if so, sell
        3. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Sell long
    
    Short:
        1. Check if symbol crosses under lower boll. band
        2. Check if close goes over stop loss; if so, sell
        3. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Sell short
"""

'\nBUY:\n    Long: \n        1. Check if symbol crosses under lower boll. band\n        2. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Buy long\n\n    Short:\n        1. Check if symbol crosses over upper boll. band\n        2. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Buy short\n\nSELL:\n    Long:\n        1. Check if symbol crosses over upper boll. band\n        2. Check if close is under stop loss; if so, sell\n        3. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Sell long\n    \n    Short:\n        1. Check if symbol crosses under lower boll. band\n        2. Check if close goes over stop loss; if so, sell\n        3. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Sell short\n'

In [6]:
env = TradingEnv(balance_amount=100, balance_unit='BUSD', symbols=symbols)
for i in range(len(df)):    
    print(env.balance_amount, env.balance_unit, env.number_of_buys)
    if env.long_unit == None and env.short_unit == None:  # Want to buy
        for symbol in symbols:

            ######## LONG           (lower + ((upper-lower)*multiplier))
            if env.cross_down[symbol] == True and (df[f'{symbol}-USD_Close'].iloc[i] <= env.good_buy('long', df[f'{symbol}_Upper_Band'].iloc[i], df[f'{symbol}_Lower_Band'].iloc[i], df[f'{symbol}-USD_Close'].iloc[i])) and (df[f'{symbol}-USD_Close'].iloc[i] >= df[f'{symbol}_SMA_200'].iloc[i]): #and df[f'{symbol}_SuperDir'].iloc[i] == 1 and df[f'{symbol}_RSI'].iloc[i] <= 30
                env.buy_long(symbol, long_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i],time=df['OpenTime'].iloc[i])
                env.reset_crosses(symbols)
                break
            
            if df[f'{symbol}-USD_Low'].iloc[i] <= df[f'{symbol}_Lower_Band'].iloc[i]:
                env.cross_down[symbol] = True
    

            ######## SHORT
            if env.cross_up[symbol] == True and (df[f'{symbol}-USD_Close'].iloc[i] >= env.good_buy('short', df[f'{symbol}_Upper_Band'].iloc[i], df[f'{symbol}_Lower_Band'].iloc[i], df[f'{symbol}-USD_Close'].iloc[i]) and (df[f'{symbol}-USD_Close'].iloc[i] <= df[f'{symbol}_SMA_200'].iloc[i])): #and df[f'{symbol}_SuperDir'].iloc[i] == -1 and df[f'{symbol}_RSI'].iloc[i] >= 70
                env.buy_short(symbol, short_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i], time=df['OpenTime'].iloc[i])
                env.reset_crosses(symbols)
                break
            
            if df[f'{symbol}-USD_High'].iloc[i] >= df[f'{symbol}_Upper_Band'].iloc[i]:
                env.cross_up[symbol] = True



    ####################################################################################################################

    ############## Want to sell long position
    elif env.long_unit != None:

        # Sell long for profit
        if env.cross_up[env.long_unit] == True and df[f'{env.long_unit}-USD_Close'].iloc[i] <= (df[f'{env.long_unit}_Upper_Band'].iloc[i] - ((df[f'{env.long_unit}_Upper_Band'].iloc[i]-df[f'{env.long_unit}_Lower_Band'].iloc[i]) * env.band_stop_run)):
            print('sell long', df[f'{env.long_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.wins += 1
            env.sell_long(sell_price=df[f'{env.long_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell long at stop loss
        elif df[f'{env.long_unit}-USD_Close'].iloc[i] <= env.sl:
            print('Stop loss hit: sell long', df[f'{env.long_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.losses += 1
            env.sell_long(sell_price=df[f'{env.long_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell long at break-even  
        if (env.sma_cross_up[env.long_unit] == True) and (df[f'{env.long_unit}-USD_Low'].iloc[i] <= env.long_price*((1/env.trading_fee_multiplier)**2)) and (df[f'{env.long_unit}-USD_High'].iloc[i] >= env.long_price*((1/env.trading_fee_multiplier)**2)):
            print('Break even hit: sell long', env.long_price*((1/env.trading_fee_multiplier)**2), df['OpenTime'].iloc[i], '\n\n')
            env.ties += 1
            env.sell_long(sell_price=env.long_price*((1/env.trading_fee_multiplier)**2), time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue

        # Check if SMA is crossed up
        if df[f'{env.long_unit}-USD_High'].iloc[i] >= df[f'{env.long_unit}_SMA'].iloc[i]:
            print('SMA long crossed')
            env.sma_cross_up[env.long_unit] = True

        # Check if upper band is crossed
        if df[f'{env.long_unit}-USD_High'].iloc[i] >= df[f'{env.long_unit}_Upper_Band'].iloc[i]:
            print('long crossed')
            env.cross_up[env.long_unit] = True
    

    ############## Want to sell short position
    else:
        print(env.short_amount)
        # Sell short for profit
        if env.cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_Close'].iloc[i] >= (df[f'{symbol}_Lower_Band'].iloc[i] + ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)):
            print('sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.wins += 1
            env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell short at stop loss
        elif df[f'{env.short_unit}-USD_Close'].iloc[i] >= env.sl:
            print('Stop loss hit: sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.losses += 1
            env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell short at break-even  
        if env.sma_cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_High'].iloc[i] >= env.short_price*(env.trading_fee_multiplier**2) and df[f'{env.short_unit}-USD_Low'].iloc[i] <= env.short_price*(env.trading_fee_multiplier**2):
            print('Break even hit: sell short', env.short_price*((env.trading_fee_multiplier)**2), df['OpenTime'].iloc[i], '\n\n')
            env.ties += 1
            env.sell_short(sell_price=env.short_price*((env.trading_fee_multiplier)**2), time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue

        # Check if SMA is crossed down
        if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_SMA'].iloc[i]:
            print('SMA short crossed')
            env.sma_cross_down[env.short_unit] = True

        # Check if lower band is crossed
        if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_Lower_Band'].iloc[i]:
            print('short crossed')
            env.cross_down[env.short_unit] = True





if env.long_unit != None:
    env.sell_long(df[f'{env.long_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
if env.short_unit != None:
    env.sell_short(df[f'{env.short_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
print(env.balance_amount, env.balance_unit, env.number_of_buys, env.wins, env.ties, env.losses)

100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0


 filled long 3808.94 2022-01-03 10:15:00
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99.96000000000001 BUSD 1
99

In [7]:

data = pd.DataFrame([(i[1] for i in env.buys), (j[2] for j in env.buys), (a[3] for a in env.buys), (k[1] for k in env.sells), (l[2] for l in env.sells)])
data

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,2022-01-03 10:15:00,2022-01-04 03:15:00,2022-01-04 13:30:00,2022-01-04 19:15:00,2022-01-05 08:15:00,2022-01-08 03:15:00,2022-01-08 04:45:00,2022-01-09 12:00:00,2022-01-10 03:45:00,2022-01-10 22:00:00,...,2022-01-19 06:15:00,2022-01-20 07:30:00,2022-01-21 02:30:00,2022-01-23 01:00:00,2022-01-23 22:45:00,2022-01-24 13:00:00,2022-01-25 10:00:00,2022-01-26 15:15:00,2022-01-27 18:00:00,2022-01-28 08:45:00
1,3808.94,3768.33,3798.07,3792.27,3803.93,3218.72,3239.38,3150.25,3143.24,3106.52,...,3107.17,3132.32,2867.92,2459.12,2457.74,2376.51,2402.21,2517.93,2415.1,2428.99
2,long,short,long,long,long,short,short,short,long,short,...,short,long,short,short,long,short,long,long,short,short
3,2022-01-03 15:15:00,2022-01-04 08:30:00,2022-01-04 18:00:00,2022-01-04 20:30:00,2022-01-05 09:15:00,2022-01-08 03:45:00,2022-01-08 08:30:00,2022-01-09 18:30:00,2022-01-10 05:30:00,2022-01-11 01:45:00,...,2022-01-19 11:00:00,2022-01-20 08:15:00,2022-01-21 05:00:00,2022-01-23 09:15:00,2022-01-24 04:15:00,2022-01-24 23:45:00,2022-01-25 10:30:00,2022-01-27 04:00:00,2022-01-27 22:00:00,2022-01-28 10:00:00
4,3683.36,3844.59,3801.11028,3795.305637,3806.974971,3216.145539,3201.98,3147.730304,3145.756102,3104.035281,...,3104.684761,3134.82736,2865.626123,2457.153097,2347.67,2374.609172,2404.132922,2437.87,2413.168306,2427.047197
